In [1]:
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
import random 
#!pip install import-ipynb
#import import_ipynb
import modelparameters as mpar
import numpy as np
#from sympy import *
%load_ext Cython
import time
import array

T 108.33078115826872


In [3]:
%%cython 
import numpy as np
cimport numpy as np
from numpy cimport int32_t
from numpy cimport float64_t
import cython

cdef extern from "math.h":
    double sin(double)
    const double pi "M_PI"
cdef float64_t aep = 1.1560
cdef float64_t app = 0.8669
cdef float64_t m_p = 1836.2
cdef float64_t E_0 = 0.6
cdef float64_t w_0 = 0.058
cdef int32_t N = 3
cdef float64_t T = 2*pi*N/w_0


cdef  float64_t length2(float64_t[:] R,  float64_t[:] r):
    return pow((R[0] - r[0]),2)  + pow((R[1] - r[1]),2) + pow((R[2] - r[2]),2)

cdef  float f_e(int i,  float64_t[:] R_1,  float64_t[:] R_2,  float64_t[:] r):
    return (R_1[i] - r[i]) / (
                pow((length2(R_1,r) + pow(aep,2)), (3 / 2))) + (
                (R_2[i] - r[i])) /  (
                pow((length2(R_2,r) + pow(aep, 2)),(3 / 2)))

cdef float f_p1(int i,  float64_t[:] R_1,  float64_t[:] R_2,  float64_t[:] r) :
        return (-(R_1[i] - r[i]) / (
                pow((length2(R_1,r) + pow(aep,2)),  (3 / 2))) + (R_1[i] - R_2[i]) / pow((length2(R_1,R_2) + pow(app,2)),(3 / 2)))


cdef float f_p2(int i,  float64_t[:] R_1,  float64_t[:] R_2,  float64_t[:] r) :
        return (-(R_2[i] - r[i]) / (
                pow((length2(R_2,r) + pow(aep,2)) , (3 / 2))) - (R_1[i] - R_2[i]) / pow((length2(R_1,R_2) + pow(app,2)),(3 / 2)))

cdef float64_t[:] dadt = np.zeros((18), dtype=np.float64)



cpdef  dif( float64_t[:] a, float64_t t) :
    #x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
    cdef float64_t [:] electron = a[:3]
    cdef float64_t [:] proton1 = a[3:6]
    cdef float64_t [:] proton2 = a[6:9]
    dadt[:3]=a[9:12]
    for i in range(3):
        dadt[i+3]=a[i+12]/m_p
        dadt[i+6]=a[i+15]/m_p
    dadt[9]=f_e(0,proton1,proton2, electron)
    dadt[10]=f_e(1,proton1,proton2, electron)
    dadt[11]=f_e(2, proton1,proton2, electron)
    dadt[12]=f_p1(0, proton1,proton2, electron)
    dadt[13]=f_p1(1, proton1,proton2, electron)
    dadt[14]=f_p1(2, proton1,proton2, electron)
    dadt[15]=f_p2(0, proton1,proton2, electron)
    dadt[16]=f_p2(1, proton1,proton2, electron)
    dadt[17]=f_p2(2, proton1,proton2, electron)
    
    return np.asarray(dadt)
cdef double E(double t):
    E = E_0 * pow((sin(pi * t / T)),2) * sin(w_0 * t)
    return E
cpdef  dif2(float64_t[:]a, float64_t t2):
    #x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
    cdef int L = 100000
    cdef float64_t [:] electron = a[:3]
    cdef float64_t [:] proton1 = a[3:6]
    cdef float64_t [:] proton2 = a[6:9]
    dadt[:3]=a[9:12]
    for i in range(3):
        dadt[i+3]=a[i+12]/m_p
        dadt[i+6]=a[i+15]/m_p
    dadt[9]=f_e(0,proton1,proton2, electron)
    dadt[10]=f_e(1,proton1,proton2, electron)
    dadt[11]=f_e(2, proton1,proton2, electron) - E(t2)
    dadt[12]=f_p1(0, proton1,proton2, electron)
    dadt[13]=f_p1(1, proton1,proton2, electron)
    dadt[14]=f_p1(2, proton1,proton2, electron)+ E(t2)
    dadt[15]=f_p2(0, proton1,proton2, electron)
    dadt[16]=f_p2(1, proton1,proton2, electron)
    dadt[17]=f_p2(2, proton1,proton2, electron)+ E(t2)
    return np.asarray(dadt)

In [6]:
a_0 =  np.array([0.0, 0.0, 0.0, 
                 0.0, 0.0, 1.0, 
                 0.0, 0.0, -1.0, 
                 0.0, 0.0, 0.0, 0.1, 0.2, 0.3, -0.1, -0.2, -0.3])
t=01.0
%time d=dif2(a_0,t)
print(d)
print(a_0)

CPU times: user 32 µs, sys: 6 µs, total: 38 µs
Wall time: 40.1 µs
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.44602984e-05
  1.08920597e-04  1.63380895e-04 -5.44602984e-05 -1.08920597e-04
 -1.63380895e-04  0.00000000e+00  0.00000000e+00 -3.24994253e-06
  0.00000000e+00  0.00000000e+00 -8.69221374e-02 -0.00000000e+00
 -0.00000000e+00  8.69286373e-02]
[ 0.   0.   0.   0.   0.   1.   0.   0.  -1.   0.   0.   0.   0.1  0.2
  0.3 -0.1 -0.2 -0.3]


In [7]:
%%timeit
d=dif(a_0,t)

7.7 µs ± 300 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
%%timeit
d=dif2(a_0,t)


7.72 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
p = mpar.p
R_1 = np.array([-mpar.dist/2, 0.0, 0.0]) #начальные положения pr1,pr2,e
R_2 = np.array([mpar.dist/2, 0.0, 0.0])
r = np.array([0,0,0])

print (mpar.dist)
def a_0(i):
    a_0 = np.array([0.0, 0.0, 0.0,
                    -mpar.dist / 2, 0.0, 0.0,
                    mpar.dist / 2, 0.0, 0.0,
                    p[i][0], p[i][1], p[i][2], 
                    0.0, 0.0, 0.0, 
                    0.0, 0.0, 0.0])
    return a_0
solution =  []
start = time.time()
#odeint(dif, a_0(i), mpar.t)
#Решения 18 уравнений для n экспериментов
for i in range (mpar.n):
    solution.append(odeint(dif, a_0(i), mpar.t))



#Решаем сиситему 18 диф. уравнений с лазером
t2 = np.linspace(mpar.u, mpar.u + 325, 100000)

def a_00(i):
    a_00 = solution[i][mpar.L -1]
    return a_00
solution2 = []


# Решения 18 уравнений для n экспериментов
for i in range(mpar.n):
    solution2.append(odeint(dif2, a_00(i), t2))

end = time.time()
vt = end-start
print('Время выполнения:',vt)
print('Среднее время выполнения одного эксперимента:',vt/mpar.nn)

1.5070016255625132
Время выполнения: 7.31968092918396
Среднее время выполнения одного эксперимента: 0.731968092918396
